In [36]:
from click import style
import numpy as np
import pandas as pd
from datetime import datetime 

from scipy import optimize
from scipy import integrate

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

sns.set(style='darkgrid')

mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

In [37]:
df_analyse = pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep = ';')
df_analyse.sort_values('date', ascending=True).head()


,date,India,US,Italy,France,Vietnam,Germany
0,2020-01-22,0,1,0,0,0,0
1,2020-01-23,0,1,0,0,2,0
2,2020-01-24,0,2,0,2,2,0
3,2020-01-25,0,2,0,3,2,0
4,2020-01-26,0,5,0,3,2,0


In [38]:

N0 = 1000000
beta = 0.4
gamma = 0.1

I0 = df_analyse.Germany[41]
S0 = N0 - I0
R0 =0



In [39]:
def SIR_model(SIR, beta, gamma):
    S,I,R = SIR
    ds_dt = -beta*S*I/N0
    dI_dt = beta*S*I/N0-gamma*I
    dR_dt = gamma*I

    return ([ds_dt, dI_dt, dR_dt])

In [40]:
def SIR_model_t(SIR, t, beta, gamma):

    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I

    return dS_dt,dI_dt,dR_dt

In [41]:
# Import libraries and make the data

from scipy import optimize
from scipy import integrate


In [42]:

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [43]:
country_list = ['Germany','Italy','France','Vietnam']


In [44]:
df_plot = pd.DataFrame()

In [45]:
for each in country_list:
    ydata = np.array(df_analyse[each].iloc[31:500])
    t=np.arange(len(ydata))
    df_plot['time'] = t
    df_plot[each] = ydata

    # Initialize the variables here
    I0=ydata[0]
    S0=N0-I0
    R0=0
    beta

    popt=[0.4,0.1]
    output = fit_odeint(t, *popt)
    print(len(output))
    
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata) # pcov returns covariance matrix (2D array) of the parameters with diagonale of the matrix as a variance values of the parameters
    perr = np.sqrt(np.diag(pcov)) # Square root of the variance is the standard deviation of the parameter values. 
        
    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

    fitted=fit_odeint(t, *popt)
    df_plot[str(each+'_fitted')] = fitted



469


/home/dell/anaconda3/envs/env/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

/home/dell/anaconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning:

overflow encountered in double_scalars

/home/dell/anaconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

overflow encountered in double_scalars

/home/dell/anaconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

overflow encountered in double_scalars



standard deviation errors :  [2.65019359e-04 4.76484054e-05]  start infect: 16
Optimal parameters: beta = 0.03922052194677955  and gamma =  -0.0062107468656026725
469
standard deviation errors :  [3.95181717e-04 6.12341807e-05]  start infect: 62
Optimal parameters: beta = 0.03712488240512462  and gamma =  -0.00644709985515448
469
standard deviation errors :  [6.18345920e-04 5.85680632e-05]  start infect: 12
Optimal parameters: beta = 0.05223206436582562  and gamma =  -0.00694481414130437
469
standard deviation errors :  [0.00137418 0.00125085]  start infect: 16
Optimal parameters: beta = 0.21322076153551253  and gamma =  0.19731326266825916


In [46]:
import plotly.graph_objects as go

fig = go.Figure()

for each in country_list:
    fig.add_trace(go.Scatter(x = df_plot.time, 
                         y = df_plot[each], 
                        mode = 'markers+lines',
                        opacity = 0.9, 
                        line_width = 1,
                        marker_size = 2,
                        name = each))

    fig.add_trace(go.Scatter(x = df_plot.time, 
                         y = df_plot[str(each+'_fitted')], 
                        mode = 'markers+lines',
                        opacity = 0.9, 
                        line_width = 1,
                        marker_size = 2,
                        name = str(each+'_fitted')))

    fig.update_layout(width =900,
                 height = 600, 
                 xaxis_title = 'Time',
                 yaxis_title = "Infected cases along with curve fitting", 
                 )



fig.show()

In [47]:
import dash
from dash import dcc
from dash import html

app = dash.Dash()
app.layout = html.Div([
                        html.Label('Selecting Countries'),
                        dcc.Dropdown(id = 'country_drop_down',
                                    options=[
                                            {'label':'Germany', 'value': 'Germany'},
                                            {'label':'Italy', 'value': 'Italy'},
                                            {'label':'France', 'value': 'France'},
                                            {'label':'Vietnam', 'value':'Vietnam'}

                                    ],
                                    value = ['Germany'],
                                    multi = True
                        ),
                        dcc.Graph(figure = fig, id = 'main_window_slope')
])

In [48]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_plot.time,
                                y=df_plot[each],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=1,
                                marker_size=2, 
                                name=each
                        )
                )

    traces.append(dict(x=df_plot.time,
                                y=df_plot[str(each+'_fitted')],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=1,
                                marker_size=2, 
                                name=str(each+'_fitted')
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                xaxis_title="Time",
                yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [49]:
import os
app.run_server(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4444)),debug = True, use_reloader = False)

Dash is running on http://0.0.0.0:4444/

Dash is running on http://0.0.0.0:4444/

Dash is running on http://0.0.0.0:4444/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [104]:
df_plot.head()

,time,Germany,Germany_fitted,Italy,Italy_fitted,France,France_fitted,Vietnam,Vietnam_fitted
0,0,16,16.000000,62,62.000000,12,12.000000,16,16.000000
1,1,16,16.743654,155,64.761029,12,12.731546,16,16.256494
2,2,16,17.521872,229,67.645009,12,13.507689,16,16.517087
3,3,16,18.336260,322,70.657413,14,14.331146,16,16.781845
4,4,21,19.188499,453,73.803960,18,15.204803,16,17.050834
